In [1]:
import os
os.environ["USE_SYMENGINE"] = "t"

from sympy.core.backend import diff, Derivative, symbols, Symbol, Function
from delierium.JanetBasis import Janet_Basis, _Dterm, LHDP
from delierium.helpers import is_function, is_derivative
from IPython.display import Math
import functools
from delierium.matrix_order import Mgrevlex,Context

print(Function)

x, y = symbols ("x y")

z = Function("z")(x,y)
diff(z,x,x)
print(locals())
w = Function("w")(x,y)
# ctx: items are in descending order
ctx_grevlex_f = Context((w,z), (x,y), Mgrevlex)
#ctx_grlex_f   = Context((w,z), (x,y), Mgrlex)
#ctx_lex_f     = Context((w,z), vars, Mlex)

f1 = diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
f2 = diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
fii = diff(z, x,x)
f3 = diff(w, x,y) - 2*x*diff(z, x,x)/y - x*diff(w,x)/y**2
#f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w/(2*y**2)
#f5 = diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)

system_2_24 = [f1,f2,f3,f4,f5]
%load_ext line_profiler


<class 'symengine.lib.symengine_wrapper.Function'>
{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'import os\nos.environ["USE_SYMENGINE"] = "t"\n\nfrom sympy.core.backend import diff, Derivative, symbols, Symbol, Function\nfrom delierium.JanetBasis import Janet_Basis, _Dterm, LHDP\nfrom delierium.helpers import is_function, is_derivative\nfrom IPython.display import Math\nimport functools\nfrom delierium.matrix_order import Mgrevlex,Context\n\nprint(Function)\n\nx, y = symbols ("x y")\n\nz = Function("z")(x,y)\ndiff(z,x,x)\nprint(locals())\nw = Function("w")(x,y)\n# ctx: items are in descending order\nctx_grevlex_f = Context((w,z), (x,y), Mgrevlex)\n#ctx_grlex_f   = Context((w,z), (x,y), Mgrlex)\n#ctx_lex_f     = Context((w,z), vars, Mlex)\n\nf1 = diff(w, y) + x*diff

NameError: name 'f4' is not defined

In [ ]:
def _init(e, context):
    from operator import mul
    from symengine import FunctionSymbol    
    if type(e) == FunctionSymbol:
        operands = [e]
    elif type(e) == Derivative:
        operands = [e]
    elif type(e) == Mul:
        operands = [e]
    elif type(e) == Symbol:
        raise ValueError(f"{e} is no term in a LHDP")
    else:
        operands = e.make_args(e)
    import pdb; pdb.set_trace()
    r = [analyze_term(context, o) for o in operands]
    dterms = {}
    for _r in r:
        dterms.setdefault(_r[0], []).append((_r[1], _r[2]))
    p = []
    for v in dterms.values():
        # v is a list of tuples
        c = 0
        for tup in v:
            c += tup[1]
        p.append(_Dterm(derivative=v[0][0], coeff=c, context=context))
    print(p)

def analyze_term(context, term):
    from operator import mul    
    operands = split_into_operands(term)
    coeffs = []
    d = []
    for operand in operands:
        if is_function(operand):
            if context.is_ctxfunc(operand):
                d.append(operand)
            else:
                coeffs.append(operand)
        elif is_derivative(operand):
            if context.is_ctxfunc(operand.args[0].func()):
                d.append(operand)
            else:
                coeffs.append(operand)
        else:
            coeffs.append(operand)
    coeffs = functools.reduce(mul, coeffs, 1)
    return str(d[0]), d[0], coeffs
def split_into_operands(term):
    if is_derivative(term):
        operands = [term]
    elif is_function(term):
        operands = [term]
    else:
        try:
            operands = term.as_ordered_factors()
        except AttributeError:
            # symengine
            operands = term.args_as_sympy()
    return operands


In [ ]:
_init(x*y, ctx_grevlex_f)
